In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

# --- CONFIGURATION ---
eh_namespace = "eh-retail-stream-tithir"
eh_name = "telemetry_stream"
#Connection String
conn_string = "Endpoint=sb://eh-retail-stream-tithir.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=zh4CpF2GknXjtj7o/JRpqJE17mPE/RGTV+AEhJzqptE="

# JAAS Config for Spark-Kafka Connector
jaas_config = f'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{conn_string}";'

kafka_options = {
    "kafka.bootstrap.servers": f"{eh_namespace}.servicebus.windows.net:9093",
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.request.timeout.ms": "60000",
    "kafka.sasl.jaas.config": jaas_config,
    "subscribe": eh_name,
    "startingOffsets": "latest"
}

# --- READ STREAM ---
df_raw = spark.readStream.format("kafka").options(**kafka_options).load()

# Define Schema
schema = StructType([
    StructField("device_id", StringType()),
    StructField("temperature", DoubleType()),
    StructField("humidity", DoubleType()),
    StructField("status", StringType()),
    StructField("timestamp", DoubleType())
])

# Parse JSON
df_parsed = (df_raw
    .select(col("value").cast("string").alias("json_payload"))
    .select(from_json(col("json_payload"), schema).alias("data"))
    .select("data.*"))

# --- WRITE TO DELTA TABLE ---
checkpoint_path = "/Volumes/azure_databricks_personal/default/volume_checkpoints/iot_spark_streaming_v2"

(df_parsed.writeStream
    .format("delta")
    .option("checkpointLocation", checkpoint_path)
    .outputMode("append")
    .toTable("azure_databricks_personal.default.iot_telemetry_realtime"))